# Decorators

In [1]:
def add(x, y):
  print(f"Function add called with {x} and {y}")
  return x + y

def multiply(x, y):
  print(f"Function multiply called with {x} and {y}")
  return x * y

print("Starting calculations...")
print("Addition Result:", add(5, 3))
print("Multiplication Result:", multiply(5, 3))  
print("Calculations completed.")

Starting calculations...


Function add called with 5 and 3


Addition Result: 8


Function multiply called with 5 and 3


Multiplication Result: 15


Calculations completed.


## Decorator for Logging

In [2]:
def log_args(func):
  def wrapper(*args, **kwargs):
    print(f"{func.__name__} called with {args} and {kwargs}")
    return func(*args, **kwargs)
  return wrapper

@log_args
def add(x, y):
  return x + y

@log_args
def multiply(x, y):
  return x * y

print("Starting calculations...")
print("Addition Result:", add(5, 3))
print("Multiplication Result:", multiply(5, 3))  
print("Calculations completed.")

Starting calculations...


add called with (5, 3) and {}


Addition Result: 8


multiply called with (5, 3) and {}


Multiplication Result: 15


Calculations completed.


## Decorator for measuring execution time

In [3]:
import time

def timer(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f"{func.__name__} took {end - start:.4f} seconds")
    return result
  return wrapper

@timer
def heavy_task():
  time.sleep(2)
  return "done"

heavy_task()

heavy_task took 2.0051 seconds


'done'

## Stacking decorator

In [6]:
@timer
@log_args
def process_data(x, y):
  time.sleep(1)
  return x + y

print(process_data(3, 4))

process_data called with (3, 4) and {}


wrapper took 1.0029 seconds


7


## Decorators with arguments

In [7]:
def conditional_log(enabled=True):
  def decorator(func):
    def wrapper(*args, **kwargs):
      if enabled:
        print(f"{func.__name__} with {args}")
      return func(*args, **kwargs)
    return wrapper
  return decorator

@conditional_log(enabled=False)
def send_email(to, msg):
  return f"Sent to {to}"

@conditional_log(enabled=True)
def send_sms(to, msg):
  return f"Sent to {to}"

print(send_email('abc@xyz.com', 'Hello'))
print(send_sms('+1234567890', 'Hello'))

Sent to abc@xyz.com


send_sms with ('+1234567890', 'Hello')


Sent to +1234567890


## Class-Based Decorators: My Favorite Trick

You can even use classes as decorators if you want to maintain state.

In [13]:
class CountCalls:
  def __init__(self, func):
    self.func = func
    self.counter = 0

  def __call__(self, *args, **kwargs):
    self.counter += 1
    print(f"Call {self.counter} to {self.func.__name__}")
    return self.func(*args, **kwargs)

@CountCalls
def greet(name):
  return f"Hello {name}"

print(greet("Alice"))
print(greet("Bob"))
print(greet.counter)

Call 1 to greet


Hello Alice


Call 2 to greet


Hello Bob


2


## Nesting, Currying, and Returning Functions

You can use decorators to return new functions dynamically.

Example: a decorator that returns different logic based on arguments.

In [14]:
def math_operation(op):
  def decorator(func):
    def wrapper(*args):
      if op == "square":
        return [x ** 2 for x in args]
      elif op == "double":
        return [x * 2 for x in args]
      return func(*args)
    return wrapper
  return decorator

@math_operation("square")
def process_numbers(*args):
  return args

print(process_numbers(1, 2, 3))

[1, 4, 9]


## Timing + Logging + Retry

In [15]:
from functools import wraps
import time

def retry(times=3, delay=1):
  def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
      for i in range(times):
        try:
          return func(*args, **kwargs)
        except Exception as e:
          print(f"Error: {e}, retrying {i+1}/{times}")
          time.sleep(delay)
      return "Failed after retries"
    return wrapper
  return decorator

@retry(times=2, delay=2)
@timer
@log_args
def risky_operation():
  raise ValueError("Boom")

risky_operation()

risky_operation called with () and {}


Error: Boom, retrying 1/2


risky_operation called with () and {}


Error: Boom, retrying 2/2


'Failed after retries'